In [1]:
import requests, re
from bs4 import BeautifulSoup

In [2]:
def url_gen(ref,servings=1, metric = 'true' ):
    tail= 'print/?recipeType=Recipe&servings={}&isMetric={}'.format(servings,metric)
    url = 'https://www.allrecipes.com/recipe/' + ref+tail
    return url

In [3]:
def get_ingredients(ref): 
    
    url = url_gen(ref)
    r=requests.get(url)
    
    ingredients = []
    soup = BeautifulSoup(r.text.encode('utf-8'),'lxml')    
    col1 = soup.find('h2').next_sibling.next_sibling
    col2 = col1.next_sibling.next_sibling

    for i in col1.findAll('li'):
        last=str(i.contents).rfind('r')
        first=str(i.contents).find('n')
        ingredients.append((str(i.contents)[(first+1):(last-1)].lstrip()))
    for i in col2.findAll('li'):
        last=str(i.contents).rfind('r')
        first=str(i.contents).find('n')
        ingredients.append((str(i.contents)[(first+1):(last-1)].lstrip()))
    return ingredients

In [4]:
def get_nutrition(ref):
#Returns a dictionnary with nutritional values

    url = 'https://www.allrecipes.com/recipe/' + ref + 'fullrecipenutrition/'
    r=requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'),'lxml') 
    info = soup.findAll('div', class_='nutrition-row')
    general = {}
    general['Title']  = str(soup.find('h2').contents[0])
    general['Reference'] = ref
    general['Region'] = 'Unknown'
    nutrition = {}
    getCal = str(soup.find('div', class_="nutrition-top light-underline"))
    key = 'Calories:</span> '
    nutrition['Calories'] = int(getCal[getCal.find(key) + len(key):getCal.rfind('<br/>')])
    
    for i in info:
        a=i.find(class_='nutrient-name')
        b = re.findall('>(.*?):', str(a))[0]
        c = re.findall('value">(.*?)<', str(a))[0]
       # print(str(b) + ' : ' + str(c))
        nutrition[b] = c
    return general, nutrition

In [5]:
class Recipe:
    def __init__(self, name):
        self.name = name
        self.ingredients = get_ingredients(name)
        self.general, self.nutrition = get_nutrition(name)
    def display(self):
        print('\n ----------------- \n')
        print(self.general)
        print('\n')
        print(self.ingredients)
        print('\n')
        print(self.nutrition)
    def toDict(self):
        dictio = {}
        dictio['General'] = self.general
        dictio['Nutrition'] = self.nutrition
        dictio['Ingredients'] = self.ingredients
        return dictio

In [6]:
ref = '246179/black-chana-with-potato/'
ref = '45736/chicken-tikka-masala/'
#get_ingredients(ref)
a = Recipe(ref)
a.toDict()

{'General': {'Title': 'Chicken Tikka Masala',
  'Reference': '45736/chicken-tikka-masala/',
  'Region': 'Unknown'},
 'Nutrition': {'Calories': 404,
  'Total Fat': '28.9g',
  'Saturated Fat': '17.0g',
  'Cholesterol': '143mg',
  'Sodium': '1592mg',
  'Potassium': '660mg',
  'Total Carbohydrates': '13.3g',
  'Dietary Fiber': '2.5g',
  'Protein': '24.6g',
  'Sugars': '7g',
  'Vitamin A': '2432IU',
  'Vitamin C': '11mg',
  'Calcium': '206mg',
  'Iron': '3mg',
  'Thiamin': '0mg',
  'Niacin': '13mg',
  'Vitamin B6': '1mg',
  'Magnesium': '57mg',
  'Folate': '24mcg'},
 'Ingredients': ['60 ml yogurt',
  '4 ml lemon juice',
  '1 g fresh ground cumin',
  '0.6 g ground cinnamon',
  '0.9 g cayenne pepper',
  '1 g freshly ground black pepper',
  '1 g minced fresh ginger',
  '2 g salt, or to taste',
  '3/4 boneless skinless chicken breasts, cut into bite-size pieces',
  '1 long skewers',
  '4 g butter',
  '1/4 clove garlic, minced',
  '1/4 jalapeno pepper, finely chopped',
  '1 g ground cumin',
  '1

In [7]:
def get_recipes(themeUrl, number = 20):
    
    r=requests.get(themeUrl) #add page 1,2,etc
    
    baseUrl = 'https://www.allrecipes.com/recipe/'
    soup = BeautifulSoup(r.text.encode('utf-8'),'lxml') 
    results = soup.findAll('h3', class_='fixed-recipe-card__h3')
    
    theme = []
    iteration = 0
    for i in results :
        
        if iteration  > number:
            break
            
        iteration = iteration + 1
        
        link = str(i.find('a').get('href'))
        first=link.find(baseUrl)
        theme.append(link[(first+len(baseUrl)):])
        
    return theme

In [13]:
themeUrl = 'https://www.allrecipes.com/recipes/233/world-cuisine/asian/indian/' 
#themeUrl = 'https://www.allrecipes.com/recipes/728/world-cuisine/latin-american/mexican/'
themeLinks = get_recipes(themeUrl, number = 20)
themeRecipes = []
themeDico = {}

from tqdm import tqdm_notebook as tqdm

for i in tqdm(range(number)):
    recipe = themeLinks[i]
#for recipe in themeLinks:
    try:
        current = Recipe(recipe)
        themeRecipes.append(current)
        themeDico[current.general['Reference']] = current.toDict()
    except AttributeError:
        print('Error')
        continue

In [14]:
import json

with open('indian.json', 'w') as f:
    json.dump(themeDico, f)

In [15]:
themeDico

{'35909/indian-pork-chops/': {'General': {'Title': 'Indian Pork Chops',
   'Reference': '35909/indian-pork-chops/',
   'Region': 'Unknown'},
  'Nutrition': {'Calories': 220,
   'Total Fat': '15.4g',
   'Saturated Fat': '4.0g',
   'Cholesterol': '36mg',
   'Sodium': '172mg',
   'Potassium': '277mg',
   'Total Carbohydrates': '7.2g',
   'Dietary Fiber': '0.5g',
   'Protein': '13.3g',
   'Sugars': '7g',
   'Vitamin A': '628IU',
   'Vitamin C': '1mg',
   'Calcium': '18mg',
   'Iron': '1mg',
   'Thiamin': '0mg',
   'Niacin': '6mg',
   'Vitamin B6': '0mg',
   'Magnesium': '16mg',
   'Folate': '3mcg'},
  'Ingredients': ['8 ml vegetable oil',
   '1 (4 ounce) pork chops',
   '45 ml apricot nectar',
   '0.5 g curry powder',
   '0.2 g garlic powder',
   '0.4 g salt']},
 '231026/keema-aloo-ground-beef-and-potatoes/': {'General': {'Title': 'Keema Aloo (Ground Beef and Potatoes)',
   'Reference': '231026/keema-aloo-ground-beef-and-potatoes/',
   'Region': 'Unknown'},
  'Nutrition': {'Calories': 362,